<a href="https://colab.research.google.com/github/tyoamazinglib/recommender-system-tfrs/blob/master/recsys_tfrs_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation Steps

In [ ]:
!pip install tensorflow_recommenders ## Essential if running in Google Colab
!pip install scann

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 3.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.5 MB 7.6 MB/s 
     |████████████████████████████████| 588.3 MB 19 kB/s 
     |████████████████████████████████| 6.0 MB 55.3 MB/s 
     |████████████████████████████████| 1.7 MB 49.0 MB/s 
     |████████████████████████████████| 439 kB 62.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
 

In [ ]:
## Basic Library Imports
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
from tensorflow import keras
from typing import Dict, Text

**Load MovieLens 1M Dataset**


---
To load the datasets, we could use the help of TensorFlow Datasets API to load our data into an Datasets Object that would act as a generator. This way we could manipulate our dataset easily using the TensorFlow datasets API whether we want to split, shuffle, or even batches our dataset in a way we desire.

In [ ]:
ratings = tfds.load("movielens/1m-ratings", split="train")
movies = tfds.load("movielens/1m-movies", split="train")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1000209 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3883 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/movielens/1m-movies/0.1.1.incomplete14JMD1/movielens-train.tfrecord*...:   0%|…

Dataset movielens downloaded and prepared to ~/tensorflow_datasets/movielens/1m-movies/0.1.1. Subsequent calls will reuse this data.


# Dataset Shuffling and Batching (Retrieval)

**Datasets Sampling**


---
There are two things we need to sample for our dataset to construct a recommendation retrieval model.


1.   Categorical features for each ratings that user has given (movie titles and the user ID itself that gives that certain ratings)
2.   A vocabulary that maps the unique list of movie ids and user ids that will be used in the embedding vectors of our model

Both will be defined in below cell

In [ ]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})

movies = movies.map(lambda x: x["movie_title"])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
# Shuffle the Ratings Dataset 
toShuffleBatch = 100_000 # Change this according to number of data will be batched from 1M instances of data
trainPercentage = 0.8
trainInstances = int(trainPercentage * toShuffleBatch)

tf.random.set_seed(42)
shuffled = ratings.shuffle(toShuffleBatch, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(trainInstances)
test = shuffled.skip(trainInstances).take(toShuffleBatch-trainInstances)

In [ ]:
# Define a vocabulary that maps unique list of movie and user ids to the embedding vectors of the model 
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie = np.unique(np.concatenate(list(movie_titles)))
unique_ids = np.unique(np.concatenate(list(user_ids)))
print(f"There are {unique_movie.shape[0]} numbers of unique movies and {unique_ids.shape[0]} numbers of unique user ids in the dataset")

There are 3883 numbers of unique movies and 6040 numbers of unique user ids in the dataset


# Base model definition

In [ ]:
# Define Embedding Dimension (arbitrary)
embedding_dimension = 64

**Defining Query and Candidate Model**


---
Query and candidate are just an another way to refers to what will be querying and what we expect to get from those query which is linearly in context of a recommender model.

In our case the query is for a specific person that has a specific unique id, what the model thinks of this person as having a set of movie recommendation that fits the user as saying there are a set of movie that this user might like, thus making it as a 'candidate'.

In [ ]:
# Modelling for each 'query' and 'candidate'

query_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(vocabulary=unique_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_ids) + 1, embedding_dimension),
])

candidate_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(vocabulary=unique_movie, mask_token=None),
    tf.keras.layers.Embedding(len(unique_movie) + 1, embedding_dimension),
])

**Metrics and Loss**


---
As we are dealing with a pairs of user-movie that match their expected ratings, we will be measure our model performance by comparing their affinity score of a positive user-movie pairs to the scores of all the other possible candidates. A score of 1 will deduct that our model are perfectly accurate and could predict all positive user-movie pairs correctly.

TFRS does provide an easy way to easily implement this metrics with the ``` tfrs.metrics.FactorizedTopK``` class.

Below are a more thorough definition of this class taken from the official documentation at [here](https://www.tensorflow.org/recommenders/api_docs/python/tfrs/metrics/FactorizedTopK):


```
tfrs.metrics.FactorizedTopK(
    candidates: Union[layers.factorized_top_k.TopK, tf.data.Dataset],
    ks: Sequence[int] = (1, 5, 10, 50, 100),
    name: str = 'factorized_top_k'
) -> None
```
Args:

*   candidates : A layer for retrieving top candidates in response to a query, or a dataset of candidate embeddings from which candidates should be retrieved.
*   ks :	A sequence of values of k at which to perform retrieval evaluation.
*   name : Optional name (defaults to 'factorized_top_k')


Loss function for retrieval model is also elegantly could be implemented using the ```Retrival``` task object that is essentially a wrapper that bundles the appropriate loss function according to the metric that we use for this specific retrieval model


In [ ]:
# Define a FactorizedTopK as our metrics using our candidate models for the layer of evaluating the candidates as well as the Loss Function
K_Factor = (5, 10, 20, 40, 60, 80, 100, 200, 400, 600, 800, 1000) # Change this according to desired values

top_k_metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(candidate_model),
  ks = K_Factor
)

Loss = tfrs.tasks.Retrieval(
  metrics=top_k_metrics
)

# Defining Retrieval Model

**Retrieval Model Definition**


---
Here, we define our retrieval model by extending a class definition from the base class of ``` tfrs.models.Model```. The base model will have all the definition regarding the training loop needed for this model to works. 

There's also an alternative way to define the same exact model by extending the default plain ``` tf.keras.Model``` and then override both of the ```train_step``` and ```test_step``` method to appropriately fits the context of a retrieval model.

In [ ]:
# Easy way to define the model using the tfrs.models.Model class as a base class

class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model, loss_function):
    super().__init__()
    self.user_model: tf.keras.Model = user_model
    self.movie_model: tf.keras.Model = movie_model
    self.loss_function: tf.keras.layers.Layer = loss_function 

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.loss_function(user_embeddings, positive_movie_embeddings)

In [ ]:
# Manually defining the model from scratch by extending the tf.keras.Model class

class NoBaseClassMovielensModel(tf.keras.Model):

  def __init__(self, user_model, movie_model, loss_function):
    super().__init__()
    self.user_model: tf.keras.Model = user_model
    self.movie_model: tf.keras.Model = movie_model
    self.loss_function: tf.keras.layers.Layer = loss_function 

  def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Set up a gradient tape to record gradients.
    with tf.GradientTape() as tape:

      # Loss computation.
      user_embeddings = self.user_model(features["user_id"])
      positive_movie_embeddings = self.movie_model(features["movie_title"])
      loss = self.loss_function(user_embeddings, positive_movie_embeddings)

      # Handle regularization losses as well.
      regularization_loss = sum(self.losses)

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

  def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Loss computation.
    user_embeddings = self.user_model(features["user_id"])
    positive_movie_embeddings = self.movie_model(features["movie_title"])
    loss = self.loss_function(user_embeddings, positive_movie_embeddings)

    # Handle regularization losses as well.
    regularization_loss = sum(self.losses)

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

# Fitting Retrieval Model

In [ ]:
# Compile the model for fitting and evaluating
retrieval_model = MovielensModel(query_model, candidate_model, Loss) # Can be changed to the NoBaseClass version of the model
retrieval_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# Shuffling train and test data. Caching will help with the performance when handling large datasets such as these.
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [ ]:
## OPTIONAL
# Loads up the TensorBoard Notebook Extension for model evaluation
%load_ext tensorboard
import datetime

log_dir_retrieval = "logs/fit_retrieval/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback_retrieval = tf.keras.callbacks.TensorBoard(log_dir=log_dir_retrieval, histogram_freq=1)

In [ ]:
# Fit the training data into the retrieval model
retrieval_model.fit(cached_train,
                    epochs=20,  
                    callbacks=[tensorboard_callback_retrieval])

Epoch 1/20
10/10 [==============================] - 204s 19s/step - factorized_top_k/top_5_categorical_accuracy: 0.0014 - factorized_top_k/top_10_categorical_accuracy: 0.0040 - factorized_top_k/top_20_categorical_accuracy: 0.0092 - factorized_top_k/top_40_categorical_accuracy: 0.0199 - factorized_top_k/top_60_categorical_accuracy: 0.0296 - factorized_top_k/top_80_categorical_accuracy: 0.0389 - factorized_top_k/top_100_categorical_accuracy: 0.0482 - factorized_top_k/top_200_categorical_accuracy: 0.0900 - factorized_top_k/top_400_categorical_accuracy: 0.1568 - factorized_top_k/top_600_categorical_accuracy: 0.2130 - factorized_top_k/top_800_categorical_accuracy: 0.2625 - factorized_top_k/top_1000_categorical_accuracy: 0.3086 - loss: 70367.1044 - regularization_loss: 0.0000e+00 - total_loss: 70367.1044
Epoch 2/20
10/10 [==============================] - 182s 18s/step - factorized_top_k/top_5_categorical_accuracy: 0.0375 - factorized_top_k/top_10_categorical_accuracy: 0.0721 - factorized_to

KeyboardInterrupt: ignored

# Evaluating Retrieval Model

In [ ]:
# Fetch top_k metrics and create a custom scalars in TensorBoard
top_k_acc = {}
for k in K_Factor:
  top_k_acc[k] = retrieval_model.history.history[f'factorized_top_k/top_{k}_categorical_accuracy']

epochs = len(retrieval_model.history.history['loss'])

from torch.utils.tensorboard import SummaryWriter
layout = {
    "Factorized Top K Losses": {
        "Top_K_Acc": ["Multiline", [f"Top_K_Acc/K={k}" for k in K_Factor]],
    },
}

writer = SummaryWriter()
writer.add_custom_scalars(layout)

for epoch in range(epochs):
    writer.add_scalars("Top_K_Acc", {f"K={k}":top_k_acc[k][epoch] for k in K_Factor}, epoch)

writer.close()

In [ ]:
# Shows TensorBoard on the retrieval model
%tensorboard --logdir=runs

In [ ]:
# Matplotlib Version of the plot
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2)
fig.suptitle('Retrieval model evaluation')
fig.set_size_inches(15, 7)

for k in K_Factor:
  ax[0].plot(range(epochs), top_k_acc[k], label=f"top_{k}")

ax[0].set_title("Accuracy")
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")
ax[0].legend()

ax[1].plot(range(epochs), retrieval_model.history.history["total_loss"], label="total_loss")
ax[1].set_title("Loss")
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")

plt.show()

In [ ]:
# Set an array of test set as batches
test_flat = np.concatenate(list(test.map(lambda x: x["user_id"]).batch(1000).as_numpy_iterator()), axis=0)

# Compute the ground truth of the model at K maximum of the trained model
ground_truth = tfrs.layers.factorized_top_k.BruteForce(
    query_model=retrieval_model.user_model,
    k=np.max(K_Factor))
ground_truth.index_from_dataset(
    tf.data.Dataset.zip((movies.batch(1000),movies.batch(1000).map(retrieval_model.movie_model))))
_, y_true = ground_truth(test_flat)

# Predict using ScaNN approximation
PREC_K = []
REC_K = []

for k in K_Factor:
  scann_approx = tfrs.layers.factorized_top_k.ScaNN(
      query_model=retrieval_model.user_model,
      k=k
  )
  scann_approx.index_from_dataset(
      tf.data.Dataset.zip((movies.batch(1000),movies.batch(1000).map(retrieval_model.movie_model))))
  _, titles = scann_approx(test_flat)
  PREC_K.append(compute_precision(y_true, titles))
  REC_K.append(compute_recall(y_true, titles))

FSCORE_K = [(2*PREC*REC)/(PREC+REC) for PREC, REC in zip(PREC_K, REC_K)]

In [ ]:
# Accuracy vs Other Metrics graph
fig, ax = plt.subplots(1, 2)
fig.suptitle('Accuracy vs P@K, R@K and F-Score@K')
fig.set_size_inches(20, 8)
X_K = np.arange(len(K_Factor))

ax[0].bar(X_K, [top_k_acc[k][-1] for k in top_k_acc], color='white', edgecolor='salmon')
ax[0].plot(X_K, [top_k_acc[k][-1] for k in top_k_acc], marker='o', color='salmon')
ax[0].set_title("Final Accuracy")
ax[0].set_xlabel("K")
ax[0].set_xticks(X_K)
ax[0].set_xticklabels(K_Factor)
ax[0].set_ylabel("Accuracy")

ax[1].bar(X_K - 0.25, PREC_K, label="PREC@K", width=0.25, color='white', edgecolor='lightcoral')
ax[1].bar(X_K, REC_K, label="REC@K", width=0.25, color='white', edgecolor='wheat')
ax[1].bar(X_K + 0.25, FSCORE_K, label="F-Score@K", width=0.25, color='white', edgecolor='lightsalmon')
ax[1].plot(X_K - 0.25, PREC_K, marker='o', color='lightcoral')
ax[1].plot(X_K, REC_K, marker='o', color='wheat')
ax[1].plot(X_K + 0.25, FSCORE_K, marker='o', color='lightsalmon')
ax[1].set_title("Final Metrics")
ax[1].set_xlabel("K")
ax[1].set_xticks(X_K)
ax[1].set_xticklabels(K_Factor)
ax[1].set_ylabel("Value")
ax[1].legend()

plt.show()

# Dataset Shuffling and Batching (Ranking)

**Datasets Sampling**


---
Ranking model has almost the same exact sampling as retrieval model with the exception of adding a numerical features ```user_rating```. 

This will be defined in below cell. The rest is almost exactly the same as retrieval model


In [ ]:
ratings = tfds.load("movielens/1m-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

In [ ]:
# Shuffle the Ratings Dataset 
toShuffleBatch = 100_000 # Change this according to number of data will be batched from 1M instances of data
trainPercentage = 0.8
trainInstances = int(trainPercentage * toShuffleBatch)

tf.random.set_seed(42)
shuffled = ratings.shuffle(toShuffleBatch, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(trainInstances)
test = shuffled.skip(trainInstances).take(toShuffleBatch-trainInstances)

In [ ]:
# Define a vocabulary that maps unique list of movie and user ids to the embedding vectors of the model 
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))
print(f"There are {unique_movie_titles.shape[0]} numbers of unique movies and {unique_user_ids.shape[0]} numbers of unique user ids in the dataset")

# Defining Ranking Model

TFRS does provide an easy way to easily implement this metrics with the ``` tfrs.metrics.FactorizedTopK``` class.

Below are a more thorough definition of this class taken from the official documentation at [here](https://www.tensorflow.org/recommenders/api_docs/python/tfrs/metrics/FactorizedTopK):


```
tfrs.metrics.FactorizedTopK(
    candidates: Union[layers.factorized_top_k.TopK, tf.data.Dataset],
    ks: Sequence[int] = (1, 5, 10, 50, 100),
    name: str = 'factorized_top_k'
) -> None
```
Args:

*   candidates : A layer for retrieving top candidates in response to a query, or a dataset of candidate embeddings from which candidates should be retrieved.
*   ks :	A sequence of values of k at which to perform retrieval evaluation.
*   name : Optional name (defaults to 'factorized_top_k')


Loss function for retrieval model is also elegantly could be implemented using the ```Retrival``` task object that is essentially a wrapper that bundles the appropriate loss function according to the metric that we use for this specific retrieval model


In [ ]:
# Define a FactorizedTopK as our metrics using our candidate models for the layer of evaluating the candidates as well as the Loss Function
rank_loss = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

**Ranking Model Definition**


---
Here, we define our retrieval model by extending a class definition from the base class of ``` tfrs.models.Model```. The base model will have all the definition regarding the training loop needed for this model to works. 

There's also an alternative way to define the same exact model by extending the default plain ``` tf.keras.Model``` and then override both of the ```train_step``` and ```test_step``` method to appropriately fits the context of a retrieval model.

In [ ]:
class RankingModel(tf.keras.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()

    # Compute embeddings for users.
    self.user_embeddings: tf.keras.Model = user_model

    # Compute embeddings for movies.
    self.movie_embeddings: tf.keras.Model = movie_model

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [ ]:
class MovielensRate(tfrs.models.Model):

  def __init__(self, loss_function):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel(query_model, candidate_model)
    self.task: tf.keras.layers.Layer = loss_function

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")
    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

# Fitting Ranking Model

In [ ]:
# Compile the model for fitting and evaluating
ranking_model = MovielensRate(rank_loss)
ranking_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# Shuffling train and test data. Caching will help with the performance when handling large datasets such as these.
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [ ]:
## OPTIONAL
# Loads up the TensorBoard Notebook Extension for model evaluation
%load_ext tensorboard
import datetime

log_dir_ranking = "logs/fit_ranking/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback_ranking = tf.keras.callbacks.TensorBoard(log_dir=log_dir_ranking, histogram_freq=1)

In [ ]:
# Fit the training data into the retrieval model
ranking_model.fit(cached_train, 
                  epochs=1000,
                  callbacks=[tensorboard_callback_ranking])

# Evaluating Ranking Model

In [ ]:
# Shows TensorBoard on the retrieval model
%tensorboard --logdir=logs/fit_ranking

In [ ]:
# Matplotlib version of the graph

RMSE_FIT = len(ranking_model.history.history["root_mean_squared_error"])
epochs_2 = [(x + 1) for x in range(RMSE_FIT)]

plt.plot(epochs_2, ranking_model.history.history["root_mean_squared_error"], label="RMSE")

plt.title("RMSE")
plt.xlabel("Epochs")
plt.ylabel("RMSE");
plt.legend()

# Evaluating whole recommender system

In [ ]:
# Helper Function to Evaluate Precision and Recall
def eval_at_k(predicted_dict, k):
  k_to_eval = dict(list(predicted_dict.items())[:k])
  num_relevant_items = sum(k_to_eval.values())
  num_item_recommend = len(k_to_eval)
  num_possible_relevant = sum(predicted_dict.values())
  return (num_relevant_items / num_item_recommend), (num_relevant_items / num_possible_relevant) 

In [ ]:
# Set an array of test set as batches
test_flat = np.unique(np.concatenate(list(test.map(lambda x: x["user_id"]).batch(1000).as_numpy_iterator()), axis=0))

relevant_threshold = 2.5 # Rating threshold where we would say this recommendation is relevant
MAX_K = max(K_Factor)
NUM_USER_TEST = 100

# Define the embedding to be used
index = tfrs.layers.factorized_top_k.BruteForce(retrieval_model.user_model)
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(1000), movies.batch(1000).map(retrieval_model.movie_model)))
)
_, titles = index(test_flat, k=MAX_K)


Mean_Average_Precision = {}
Mean_Average_Recall = {}

for idx, user in enumerate(test_flat[:10]):
  relevant_movies = {}
  for movie_title in list(titles[idx].numpy()):
    relevant_movies[movie_title] = True if ranking_model({
    "user_id": np.array([f"{idx}"]),
    "movie_title": np.array([movie_title])
    }) >= relevant_threshold else False
  for K in K_Factor:
    if K not in Mean_Average_Precision:
      Mean_Average_Precision[K] = []
    Mean_Average_Precision[K].append(eval_at_k(relevant_movies, K)[0])
    if K not in Mean_Average_Recall:
      Mean_Average_Recall[K] = []
    Mean_Average_Recall[K].append(eval_at_k(relevant_movies, K)[1])

for K in K_Factor:
  Mean_Average_Precision[K] = np.mean(Mean_Average_Precision[K])
  Mean_Average_Recall[K] = np.mean(Mean_Average_Recall[K])

In [ ]:
# Accuracy vs Other Metrics graph
fig, ax = plt.subplots(1, 2)
fig.suptitle('P@K vs R@K vs F-Score@K')
fig.set_size_inches(20, 5)
X_K = [str(K) for K in K_Factor]

ax[0].bar(X_K, Mean_Average_Precision.values(), color='white', edgecolor='salmon')
ax[0].plot(X_K, Mean_Average_Precision.values(), marker='o', color='salmon')
ax[0].set_title("MAP@K")
ax[0].set_xlabel("K")
ax[0].set_xticklabels(K_Factor)
ax[0].set_ylabel("MAP")

ax[1].bar(X_K, Mean_Average_Recall.values(), color='white', edgecolor='lightcoral')
ax[1].plot(X_K, Mean_Average_Recall.values(), marker='o', color='lightcoral')
ax[1].set_title("MAR@K")
ax[1].set_xlabel("K")
ax[1].set_xticklabels(K_Factor)
ax[1].set_ylabel("MAP")

plt.show()

In [ ]:
index = tfrs.layers.factorized_top_k.BruteForce(retrieval_model.user_model)

index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(1000), movies.batch(1000).map(retrieval_model.movie_model)))
)
id = 200
atK = 1000
_, titles = index(tf.constant([f'{id}']), k=atK)
print(f"Recommendations for user {id} : {titles[0, :atK]}\n")

test_ratings = {}
for movie_title in list(titles.numpy()[0]):
  test_ratings[movie_title] = ranking_model({
      "user_id": np.array([f"{id}"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score[0]}")